# Imports

In [ ]:
from os import listdir
from os.path import isfile, join
import undetected_chromedriver as uc
from selenium import webdriver
import pandas as pd

# Download products'links in a list of category webpage of Carrefour

In [ ]:
%%time

with open("pages_category.txt") as file:
    for url in file:
        options = webdriver.ChromeOptions()
        options.add_argument('headless');
        driver = uc.Chrome(chrome_options=options)
        driver.get(url.replace("\n",""))
        html = driver.page_source
        if not "Cloudflare" in html:
            soup = BeautifulSoup(html, "html.parser")
            links = set()
            for link in soup.findAll('a'):
                if "p/" in link.get('href'):
                    links.add('https://www.carrefour.fr'+link.get('href'))

            for l in links:
                with open("output_scrapping/"+url.split("/")[-1].replace("\n","")+".txt", "a+") as file_son:
                    file_son.write(l+"\n")
                    
        else:
            print(url, "Cloudflare")
            
        driver.close()

# Download content of this products

In [ ]:
%%time

onlyfiles = [f for f in listdir("output_scrapping/") if isfile(join("output_scrapping/", f))]
titles = []
urls = []
texts = []
categories = []
options = webdriver.ChromeOptions()
options.add_argument('headless');
        
for name_file in onlyfiles:
    cat = name_file.replace(".txt", "")
    with open("output_scrapping/"+name_file) as file:
        for url in file:
            url = url.replace("\n","")
            driver = uc.Chrome(chrome_options=options)
            driver.get(url)
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            if not("Cloudflare" in soup.title.text):
                text = finalpreprocess(soup.title.text).replace("carrefour", "").replace("prix","")
                titles.append(soup.title.text)
                urls.append(url)
                texts.append(text)
                categories.append(cat)            
            driver.close()

In [ ]:
df_products = pd.DataFrame()
df_products["title"] = titles
df_products["url"] = urls
df_products["text"] = texts
df_products["categories"] = categories

In [ ]:
df_products.head()

In [ ]:
df_products.shape

In [ ]:
df_products.to_csv("all_products.csv", index=False)